In [1]:
!pip3 install spacy-langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

import os

import glob
import json

from textblob import TextBlob as tb

# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
import time

import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

In [4]:
spacy.cli.download('es_core_news_sm')

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [5]:
#Declara um Staticmethod, fazendo com que seja retornado um objeto da classe Language
@Language.factory("language_detector")
#Cria a função para que seja voltado o objeto "LanguageDetector()"
def get_lang_detector(nlp, name):
   return LanguageDetector()

In [6]:
def LangDetector(text_nlp):
    #Carrega modelo de leitura de detecção de linguagem
    nlp = spacy.load('es_core_news_sm')
    #Cria um pipeline com a função definida
    nlp.add_pipe('language_detector',last=True)
    text = text_nlp
    doc = nlp(text)
    return doc._.language['language'],doc._.language['score']

In [7]:
def read_json_file(file):
    with open(file, "r", encoding='utf8') as r:
        response = r.read()
        response = response.replace('\n', '')
        response = response.replace('}{', '},{')
        response = "[" + response + "]"
        return json.loads(response)

In [8]:
#função de diminuição de dimensionalidade de listas
def flatten_list(_2d_list):
    flat_list = []
    for element in _2d_list:
        if type(element) is list:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [9]:
caminho_EQ = r"/content/drive/MyDrive/data_extraction/EQUADOR/DATA"
list_EQ = glob.glob(os.path.join(caminho_EQ, '*/*'))

# extraindo json e criando lista
dict_EQ = list()
for i in range(len(list_EQ)):
    dict_EQ.append(read_json_file(list_EQ[i]))
dict_EQ = flatten_list(dict_EQ)

In [10]:
#Extração do "place_id" pEQa coleta de dados geográficos
data_extraction = []
for i in range(len(dict_EQ)):
    try:
        for j in range(len(dict_EQ[i]['data'])):
            place_id = dict_EQ[i]['data'][j]['geo']['place_id']
            data_extraction.append(place_id) 
    except:
        continue  
data_extraction = list(set(data_extraction))

In [11]:
# # Bot de extração de dados geográficos
# # Para execução é necessário tirar os comentários do código
# options = webdriver.ChromeOptions() #Execução do webdriver do google
# options.add_experimental_option("prefs", {
#   "download.default_directory": r"E:\Blue\Bootcamp 2\github\bootcamp_covid\data_extraction\EQUADOR\GEO"
#   })
# browser = webdriver.Chrome(options=options)
# browser.get('https://twitter.com/')
# time.sleep(2)
# # Os elementos serão acessados via xpath, que é o path do HTML do site, buscando botões e caixa de interação
# browser.find_element('xpath', '/html/body/div[1]/div/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div[1]/a/div/span/span').click()
# time.sleep(2)
# # É seguro inserir seus dados, visto que o algoritmo só guardará seus dados na memória do seu computador
# login = input('Insira seu nome no twitter (tag)') #Colocar como input
# password = input('Insira sua senha') #Colocar como input
# browser.find_element('xpath', '//html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[5]/label/div/div[2]/div/input').send_keys(login)
# time.sleep(2)
# browser.find_element('xpath', '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[6]/div').click()
# time.sleep(2)
# browser.find_element('xpath', '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[3]/div/label/div/div[2]/div[1]/input').send_keys(password)
# time.sleep(2)
# browser.find_element('xpath', '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div/span/span').click()
# time.sleep(2)
# # Um loop para acessar os dados do "place_id" que estão dentro da lista data_extraction
# for place_id in data_extraction:  
#   browser.get(f'https://api.twitter.com/1.1/geo/id/{place_id}.json')
# # A API do twitter restringe o uso da coleta de "place_id" em 100 vezes/hora então esse tempo é calculado
# # prevendo possíveis "bugs" e considerando o tempo limite por hora
#   time.sleep(40)

In [12]:
path = os.listdir('/content/drive/MyDrive/data_extraction/EQUADOR/GEO')
dict_EQ_GEO = list()
for i in path:
    with open(f'/content/drive/MyDrive/data_extraction/EQUADOR/GEO/{i}', 'r', encoding='utf8') as f:
        dict_EQ_GEO.append(json.load(f))

In [23]:
# A estrutura da lista funciona com o seguinte padrão:
# O primeiro objeto (dict_AR[i]) é o tweet de várias pessoas que falaram sobre o assunto covid naquele dia
# Dentro do objeto citado anteriormente podemos acessar as seguintes informações:
######### data #########
# Inclui os dados do tweet em si 
######### includes #########
# Inclui os dados do usuário que tweetou
######### meta #########
# Inclui informações sobre a pesquisa
# Com isso, as informações que serão coletadas serão:
# - Texto
# - Data
# - Quantidade de Retweet
# - Quantidade de likes
# - Grau inicial de polaridade (Utilizando a biblioteca TextBlob)
# O grau inicial será levado como consideração como baseline inicial
 

all_text = []
for i in range(len(dict_EQ)):
    try:
        for j in range(len(dict_EQ[i]['data'])):
            text = dict_EQ[i]['data'][j]['text']
            for tweet in [dict_EQ[i]['data'][j]['text']]:
                analysis = tb(tweet)
                polarity = analysis.sentiment.polarity
            retweet = dict_EQ[i]['data'][j]['public_metrics']['retweet_count']
            like = dict_EQ[i]['data'][j]['public_metrics']['like_count']
            date = dict_EQ[i]['data'][j]['created_at'].split('T')[0]
            for z in range(len(dict_EQ_GEO)):
                if dict_EQ_GEO[z]['id'] == dict_EQ[i]['data'][j]['geo']['place_id']:
                    lat = dict_EQ_GEO[z]['centroid'][1]
                    long = dict_EQ_GEO[z]['centroid'][0]
            all_text.append(dict(text = text,
                                score = polarity,
                                retweet = retweet,
                                like = like,
                                date = date,
                                lat = lat,
                                long = long
                                )) 
    except:
        continue  

In [25]:
EQ_df = pd.DataFrame(all_text)


In [15]:
EQ_df = pd.DataFrame(all_text)
Lang_score = [LangDetector(i) for i in EQ_df['text']]
# EQ_df.to_csv('EQ_data.csv')

In [30]:
Lang_score

[('es', 0.999997393047874),
 ('es', 0.9999943432650034),
 ('es', 0.9999974454697989),
 ('es', 0.9999964441523217),
 ('es', 0.9999936754415225),
 ('es', 0.999996102628116),
 ('es', 0.9999938163685141),
 ('es', 0.999996314279441),
 ('es', 0.9999945344131904),
 ('es', 0.999998204873182),
 ('es', 0.999993450821681),
 ('es', 0.9999960022753321),
 ('es', 0.71428515660849),
 ('es', 0.9999961263840353),
 ('es', 0.9999959171935966),
 ('it', 0.9999944886380059),
 ('es', 0.9999966245807631),
 ('es', 0.9999963775926807),
 ('es', 0.9999966499072745),
 ('en', 0.9999960960190559),
 ('es', 0.9999967516404462),
 ('es', 0.999995895372327),
 ('es', 0.9999961825521302),
 ('es', 0.9999962927617572),
 ('es', 0.99999537059215),
 ('es', 0.5714292706457742),
 ('es', 0.9999965506991989),
 ('es', 0.9999949370365654),
 ('es', 0.9999951871875195),
 ('es', 0.9999981592952183),
 ('es', 0.999996084355977),
 ('es', 0.9999966804474796),
 ('es', 0.9999947477393435),
 ('es', 0.9999951599035417),
 ('es', 0.999996148179140

In [18]:
len(EQ_df)

34587

In [38]:
[Lang_score[i][0] for i in range(len(Lang_score))]

['es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'it',
 'es',
 'es',
 'es',
 'en',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'en',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'pt',
 'es',
 'es',
 'es',
 'UNKNOWN',
 'es',
 'es',
 'es',
 'UNKNOWN',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'en',
 'es',
 'es',
 'pt',
 'es',
 'es',
 'es',
 'es',
 'es',
 'ca',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'pt',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'es',
 'e

In [33]:
lang = [i for i in Lang_score[i][0]]

In [34]:
lang

['e', 'n']

In [40]:
EQ_df['lang'], EQ_df['text_simm'] = [Lang_score[i][0] for i in range(len(Lang_score))], [Lang_score[i][1] for i in range(len(Lang_score))]

In [41]:
EQ_df.head()

,text,score,retweet,like,date,lat,long,lang,text_simm
0,Recuerda que:1 casa une y 2 casas dividen.1 ca...,0.0,46,185,2021-12-31,-2.261113,-80.911950,es,0.999997
1,"Cierro el 2021 con covid , pero feeeeeliz por ...",0.0,0,4,2021-12-31,-3.314031,-79.943569,es,0.999994
2,Tengan feliz año nuevo con covid o sin agradez...,0.0,0,1,2021-12-31,-2.595117,-80.425516,es,0.999997
3,"2021: ᑌᑎ ᗷᑌƐᑎ ᗩñO\n\nmi familia completa, unid...",0.0,0,2,2021-12-31,-2.351703,-80.196906,es,0.999996
4,"@malulyoliva @pharmacys La entiendo, 🤞🤞🤞de un ...",0.0,0,1,2021-12-31,-0.166625,-78.507588,es,0.999994


In [42]:
EQ_df.to_csv('EQ_data_raw.csv')